In [1]:
# !pip install pyspark

# **1. Prepare Pyspark**

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName("MyApp").getOrCreate()


# **2. Load dataset**

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
"""
load models
"""
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [5]:
"""
load data
load the dataset to google Drive. Then copy the link of the data file
"""
data = spark.read.format("libsvm").load("drive/MyDrive/Colab Notebooks/dataset.txt")


In [6]:
data.select("features").show(1, False)


+-------------------------------------------------+
|features                                         |
+-------------------------------------------------+
|(4,[0,1,2,3],[-0.222222,0.5,-0.762712,-0.833333])|
+-------------------------------------------------+
only showing top 1 row



In [7]:
data.dtypes


[('label', 'double'), ('features', 'vector')]

In [8]:
"""
label indexer 
map a string column of labels to an ML column of label indices
"""
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)


In [9]:
"""
class for indexing categorical feature columns in a dataset of Vector
"""
featureIndexer = VectorIndexer(
    inputCol="features", outputCol="indexedFeatures", maxCategories=4
).fit(data)


In [10]:
"""
split dataset to training and testing 
"""
(trainingData, testData) = data.randomSplit([0.7, 0.3], 1)


In [11]:
def train_data(model, train, test):
    """This function combine all steps in a pipeline, then use the model to fit and return the predictd values"""
    # Combine steps to a pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, model])
    # Train data
    full_model = pipeline.fit(train)
    # Model predicts data
    preds = full_model.transform(test)
    # Print model architecture
    print(full_model.stages[2])
    # Show the first five predicted values
    preds.show(5)
    return preds


def evaluate(preds):
    """This function prints some classification metrics"""
    # Accuracy
    acc_evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel",
        predictionCol="prediction",
        metricName="accuracy",
    )
    acc = acc_evaluator.evaluate(preds)
    print("Accuracy: " + str(acc))

    # Precision
    pr_evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel",
        predictionCol="prediction",
        metricName="weightedPrecision",
    )
    precision = pr_evaluator.evaluate(preds)
    print("Precision: " + str(precision))

    # Recall
    recall_evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="weightedRecall"
    )
    recall = recall_evaluator.evaluate(preds)
    print("Recall: " + str(recall))

    # F1
    f1_evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="f1"
    )
    f1 = f1_evaluator.evaluate(preds)
    print("F1 score: " + str(f1))


# **2. Decision Tree**
Run below codes and answer question 1.

reference:

model:
https://spark.apache.org/docs/latest/mllib-decision-tree.html

evaluation:
https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#multiclass-classification

## **Model and Evaluation**
You finish codes on the f1 and recall parts and run the code. Answer the question 1.

In [12]:
# Import model
from pyspark.ml.classification import DecisionTreeClassifier

evaluate(
    train_data(
        DecisionTreeClassifier(
            maxDepth=2, featuresCol="indexedFeatures", labelCol="indexedLabel"
        ),
        trainingData,
        testData,
    )
)


DecisionTreeClassificationModel: uid=DecisionTreeClassifier_4d74e5f0432e, depth=2, numNodes=5, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------+-----------------+----------+
|label|            features|indexedLabel|     indexedFeatures| rawPrediction|      probability|prediction|
+-----+--------------------+------------+--------------------+--------------+-----------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[32.0,0.0,0.0]|    [1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[32.0,0.0,0.0]|    [1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[32.0,0.0,0.0]|    [1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[32.0,0.0,0.0]|    [1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[3.0,0.0,37.0]|[0.075,0.0,0.925]|       2.0|
+-----+--------------

## **Tweak the model**

In [13]:
evaluate(
    train_data(
        DecisionTreeClassifier(
            maxDepth=1, featuresCol="indexedFeatures", labelCol="indexedLabel"
        ),
        trainingData,
        testData,
    )
)
'''This model uses maxdepth 1, which is too shallow to distinguish different 
classes. As a result, the perfomance is so bad'''

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_327a104638dc, depth=1, numNodes=3, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+---------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|  rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+---------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[35.0,0.0,37.0]|[0.48611111111111...|       2.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[35.0,0.0,37.0]|[0.48611111111111...|       2.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[35.0,0.0,37.0]|[0.48611111111111...|       2.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[35.0,0.0,37.0]|[0.48611111111111...|       2.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[35.0,0.0,37.0]|[0.48611111111111...| 

'This model uses maxdepth 1, which is too shallow to distinguish different classes. As a result, the metrics are so bad'

In [14]:
evaluate(
    train_data(
        DecisionTreeClassifier(
            maxDepth=6, featuresCol="indexedFeatures", labelCol="indexedLabel"
        ),
        trainingData,
        testData,
    )
)
'''This model uses maxdepth 6, which means the tree can go deep and 
classify the samples that are close together'''

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_81508b280e23, depth=6, numNodes=15, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------+-------------+----------+
|label|            features|indexedLabel|     indexedFeatures| rawPrediction|  probability|prediction|
+-----+--------------------+------------+--------------------+--------------+-------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[32.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...| [0.0,0.0,2.0]|[0.0,0.0,1.0]|       2.0|
+-----+--------------------+------------+-----------

'This model uses maxdepth 6, which means the tree can go deep and classify the samples that are close together'

# **3. Random forest**
Run below codes and answer question 2.

reference:

model:
https://spark.apache.org/docs/latest/mllib-ensembles.html#random-forests 

evaluation:
https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#multiclass-classification

## **Model and Evaluation**
You finish codes on the precision and recall parts and run the code. Answer the question 2.

In [15]:
from pyspark.ml.classification import RandomForestClassifier

# Initialize an object of random forest classifier
evaluate(
    train_data(
        RandomForestClassifier(
            maxDepth=2, featuresCol="indexedFeatures", labelCol="indexedLabel"
        ),
        trainingData,
        testData,
    )
)


RandomForestClassificationModel: uid=RandomForestClassifier_de67016cbdc9, numTrees=20, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[13.8624541102983...|[0.69312270551491...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[13.8624541102983...|[0.69312270551491...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[16.3731002989050...|[0.81865501494525...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[16.3731002989050...|[0.81865501494525...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[7.17151810

## **Tweak the model**

In [16]:
# Initialize an object of random forest classifier
evaluate(
    train_data(
        RandomForestClassifier(
            maxDepth=6, featuresCol="indexedFeatures", labelCol="indexedLabel"
        ),
        trainingData,
        testData,
    )
)
'''This model uses maxdepth 6. However, the metrics are still the same. 
My conjecture is that because Random Forest is a bagging model, 
it averages numerous different trees'''

RandomForestClassificationModel: uid=RandomForestClassifier_b0d8e13f9ce7, numTrees=20, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------+---------------+----------+
|label|            features|indexedLabel|     indexedFeatures| rawPrediction|    probability|prediction|
+-----+--------------------+------------+--------------------+--------------+---------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[19.0,0.0,1.0]|[0.95,0.0,0.05]|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[19.0,0.0,1.0]|[0.95,0.0,0.05]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[17.0,0.0,3.0]|[0.85,0.0,0.15]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[17.0,0.0,3.0]|[0.85,0.0,0.15]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[7.0,0.0,13.0]|[0.35,0.0,0.65]|       2.0|
+-----+--------------------+------------+----

'This model uses maxdepth 6. However, the metrics are still the same. My conjecture is that because Random Forest is a bagging model, it averages numerous different trees'

In [17]:
# Initialize an object of random forest classifier
evaluate(
    train_data(
        RandomForestClassifier(
            maxDepth=20, featuresCol="indexedFeatures", labelCol="indexedLabel"
        ),
        trainingData,
        testData,
    )
)
'''This model uses maxdepth 20 but the result is as same as 2. 
The reason is similar to above. This model is a promising candidate 
due to its performance stability'''

RandomForestClassificationModel: uid=RandomForestClassifier_c7b4f18cfef8, numTrees=20, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------+---------------+----------+
|label|            features|indexedLabel|     indexedFeatures| rawPrediction|    probability|prediction|
+-----+--------------------+------------+--------------------+--------------+---------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[19.0,0.0,1.0]|[0.95,0.0,0.05]|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[19.0,0.0,1.0]|[0.95,0.0,0.05]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[17.0,0.0,3.0]|[0.85,0.0,0.15]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[17.0,0.0,3.0]|[0.85,0.0,0.15]|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[7.0,0.0,13.0]|[0.35,0.0,0.65]|       2.0|
+-----+--------------------+------------+----

'This model uses maxdepth 20 but the result is as same as 2. The reason is similar to above. This model is a promising candidate due to its performance stability'

# **4. Naive bayes**
Run below codes and answer question 3.

reference:

model:
https://en.wikipedia.org/wiki/Naive_Bayes_classifier

evaluation:
https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#multiclass-classification

## **Model and Evaluation**
You finish codes on the accurancy and f1 parts and run the code. Answer the question 3.

In [18]:
from pyspark.ml.classification import NaiveBayes

# Initialize an object of NB classifier
evaluate(
    train_data(
        NaiveBayes(
            smoothing=1.0,
            modelType="gaussian",
            featuresCol="indexedFeatures",
            labelCol="indexedLabel",
            thresholds=[0.5, 0.5, 0.5],
        ),
        trainingData,
        testData,
    )
)

NaiveBayesModel: uid=NaiveBayes_2a01ea38fab8, modelType=gaussian, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[3.13395250343589...|[0.98821889013074...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[2.47992991282332...|[0.99999972114088...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[2.83017174990633...|[0.98659011250279...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[3.17832324917874...|[0.93656571239189...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[2.23506599993514...|[0.40502860

## **Tweak the model**

In [19]:
# Initialize an object of NB classifier
evaluate(
    train_data(
        NaiveBayes(
            smoothing=1.0,
            modelType="gaussian",
            featuresCol="indexedFeatures",
            labelCol="indexedLabel",
            thresholds=[0.6, 0.7, 0.5],
        ),
        trainingData,
        testData,
    )
)
'''I change the threshold to be skewed but the metric values are as same as 
the original one. It means that the probability of the 
correctly predicted values is dominant'''

NaiveBayesModel: uid=NaiveBayes_e0b2f888ca67, modelType=gaussian, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[3.13395250343589...|[0.98821889013074...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[2.47992991282332...|[0.99999972114088...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[2.83017174990633...|[0.98659011250279...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[3.17832324917874...|[0.93656571239189...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[2.23506599993514...|[0.40502860

'I change the threshold to be skewed but the metric values are as same as the original one. It means that the probability of the correctly predicted values is dominant'

In [20]:
# Initialize an object of NB classifier
evaluate(
    train_data(
        NaiveBayes(
            smoothing=1.0,
            modelType="gaussian",
            featuresCol="indexedFeatures",
            labelCol="indexedLabel",
            thresholds=[0.3, 0.3, 0.8],
        ),
        trainingData,
        testData,
    )
)
'''This threshold yields better result than the default values of all 0.5. 
The explanation is that the weight of each class is not the same. A search tool
may be helpful in finding the best threshold'''

NaiveBayesModel: uid=NaiveBayes_4a4a88d79841, modelType=gaussian, numClasses=3, numFeatures=4
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|         probability|prediction|
+-----+--------------------+------------+--------------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[3.13395250343589...|[0.98821889013074...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[2.47992991282332...|[0.99999972114088...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[2.83017174990633...|[0.98659011250279...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[3.17832324917874...|[0.93656571239189...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[2.23506599993514...|[0.40502860

'This threshold yields better result than the default values of all 0.5. \nThe explanation is that the weight of each class is not the same. A search tool\nmay be helpful in finding the best threshold'

# **5. SVM**
Run below codes and answer question 4.

reference:

model:
https://en.wikipedia.org/wiki/Naive_Bayes_classifier

evaluation:
https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#multiclass-classification

## **Model and Evaluation**
You finish codes on the accurancy and precision parts and run the code. Answer the question 4.

In [21]:
import numpy
from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(
    maxIter=2, regParam=0.1, featuresCol="indexedFeatures", labelCol="indexedLabel"
)
ovr = OneVsRest(classifier=lsvc)
# Initialize an object of SVM classifier
evaluate(
    train_data(
        ovr,
        trainingData,
        testData,
    )
)


OneVsRestModel_28a2e1c669b7
+-----+--------------------+------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|prediction|
+-----+--------------------+------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[0.50965618389471...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[0.77006103896559...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.67944643362179...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.59878726254498...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.47692624838688...|       0.0|
+-----+--------------------+------------+--------------------+--------------------+----------+
only showing top 5 rows

Accuracy: 0.725
Precision: 0.8413461538461539
Recall: 0.725
F1 score: 0.6610569105691056


## **Tweak the model**

In [22]:
import numpy
from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(
    maxIter=20, regParam=0.1, featuresCol="indexedFeatures", labelCol="indexedLabel"
)
ovr = OneVsRest(classifier=lsvc)
# Initialize an object of SVM classifier
evaluate(
    train_data(
        ovr,
        trainingData,
        testData,
    )
)
'''The maxIter is set to be 20, 10 times than the original one. This will give 
the model more time to adjust the parameters to fit the data'''

OneVsRestModel_e1be58562a88
+-----+--------------------+------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|prediction|
+-----+--------------------+------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[0.31747246954499...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[0.99752083949762...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.39601565117431...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.19364475744896...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[-0.1469144356747...|       0.0|
+-----+--------------------+------------+--------------------+--------------------+----------+
only showing top 5 rows

Accuracy: 0.85
Precision: 0.8928571428571429
Recall: 0.85
F1 score: 0.8400000000000001


''

In [23]:
from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(
    maxIter=100, regParam=0.1, featuresCol="indexedFeatures", labelCol="indexedLabel"
)
ovr = OneVsRest(classifier=lsvc)
# Initialize an object of SVM classifier
evaluate(
    train_data(
        ovr,
        trainingData,
        testData,
    )
)
'''The maxIter is set to be 100, 5 times than the second one. However, the 
performance is not significantly better even though all is. This may be an 
indication that the model is reaching the best iteration before overfitting.'''

OneVsRestModel_9f4581c29520
+-----+--------------------+------------+--------------------+--------------------+----------+
|label|            features|indexedLabel|     indexedFeatures|       rawPrediction|prediction|
+-----+--------------------+------------+--------------------+--------------------+----------+
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[0.31690184047142...|       0.0|
|  0.0|(4,[0,1,2,3],[-0....|         0.0|(4,[0,1,2,3],[-0....|[0.99913574582145...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.38261860187360...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[0.18064027129859...|       0.0|
|  0.0|(4,[0,1,2,3],[0.1...|         0.0|(4,[0,1,2,3],[0.1...|[-0.1586322996944...|       0.0|
+-----+--------------------+------------+--------------------+--------------------+----------+
only showing top 5 rows

Accuracy: 0.875
Precision: 0.90625
Recall: 0.875
F1 score: 0.8690476190476191
